---
title: "Graded Problem Set: Investment and Portfolio Management"
subtitle: "Imperial College London - Business School"
author: 
  - name: "Rodolphe Lajugie"
date: "2025-10-31"
format: pdf
fontsize: 12pt
geometry: margin=1in
titlepage: true
titlepage-geometry: "top=2cm, bottom=2cm, left=2cm, right=2cm"
---
\newpage
\tableofcontents
\listoffigures
\newpage
\listoftables
\newpage

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [19]:
df = pd.read_excel('../data/data_coursework1_Q1.xlsx')
df = df[['year_', 'month_', 'date_', '1-month Tbill', 'SP500', 'IBM']]

# Calculate returns
df['SP500_ret'] = df['SP500'].pct_change()
df['IBM_ret'] = df['IBM'].pct_change()

# Adjust T-bill to monthly rate and scale (if needed)
df['rft'] = df['1-month Tbill'] / 100

df = df.dropna(subset=['SP500_ret', 'IBM_ret', 'rft'])

# Excess returns
df['excess_stock'] = df['IBM_ret'] - df['rft']
df['excess_market'] = df['SP500_ret'] - df['rft']
df['date'] = pd.to_datetime(df['year_'].astype(
    str) + '-' + df['month_'].astype(str), format='%Y-%m')
df.drop(columns=['year_', 'month_', 'date_'], inplace=True)
df

,1-month Tbill,SP500,IBM,SP500_ret,IBM_ret,rft,excess_stock,excess_market,date
25,0.20,70.22,2.66,0.016650,-0.011152,0.0020,-0.013152,0.014650,1962-02-01
26,0.20,70.29,2.64,0.000997,-0.007519,0.0020,-0.009519,-0.001003,1962-03-01
27,0.22,68.05,2.25,-0.031868,-0.147727,0.0022,-0.149927,-0.034068,1962-04-01
28,0.24,62.99,1.95,-0.074357,-0.133333,0.0024,-0.135733,-0.076757,1962-05-01
29,0.20,55.63,1.68,-0.116844,-0.138462,0.0020,-0.140462,-0.118844,1962-06-01
...,...,...,...,...,...,...,...,...,...
367,0.66,330.70,19.89,-0.081389,-0.076173,0.0066,-0.082773,-0.087989,1990-08-01
368,0.60,315.40,20.77,-0.046265,0.044243,0.0060,0.038243,-0.052265,1990-09-01
369,0.68,307.10,20.60,-0.026316,-0.008185,0.0068,-0.014985,-0.033116,1990-10-01
370,0.57,315.20,22.43,0.026376,0.088835,0.0057,0.083135,0.020676,1990-11-01


# Model 1: Simple CAPM Regression

## Part 1: linear regression
### Manual computation

We know that $\hat{\beta} = (X'X)^{-1}X'y$ <br>
Where X is the design matrix and y the response variable.


In [3]:
y = df['excess_stock'].values
X = np.column_stack((np.ones(len(df)), df['excess_market'].values))

In [4]:
beta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)
alpha, beta = beta_hat
print('alpha:', alpha, 'beta:', beta)

alpha: 0.0027020468479849896 beta: 0.8856018941150132


### With builtin functions

In [5]:
X1 = sm.add_constant(df['excess_market'])
model1 = sm.OLS(df['excess_stock'], X1).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:           excess_stock   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.284
Method:                 Least Squares   F-statistic:                     138.4
Date:                Tue, 18 Nov 2025   Prob (F-statistic):           4.24e-27
Time:                        13:26:18   Log-Likelihood:                 543.44
No. Observations:                 347   AIC:                            -1083.
Df Residuals:                     345   BIC:                            -1075.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0027      0.003      0.993

Using the OLS function from the statsmodels library or by computing it manually, we obtain the following results for the CAPM regression of IBM's excess returns against the market's excess returns:
- Intercept ($\alpha$): 0.0027
- Slope ($\beta$): 0.8856

$\alpha$ is close to zero which means IBM doesn't outperform or underperform the market on average. The $\beta$ of 0.89 indicates that IBM is less volatile than the market.

## Part 2: 2nd linear regression

We want to find $\alpha$, $\beta_1$, $\beta_2$ and $\beta_3$ such that: <br>
$$r_{it}-r_{ft} = \alpha + \beta_1 [D_t(r_{mt}-r_{ft})] + \beta_2[(1-D_t)(r_{mt}-r_{ft})] + \beta_3 (r_{mt}-r_{ft})^2 + u_t$$

Where $D_t = 1 $ if the market excess return is positive and 0 otherwise.

### Manual computation

In [6]:
Dt = (df['excess_market'] > 0).astype(int).values
D_excess = Dt * df['excess_market'].values
nonD_excess = (1 - Dt) * df['excess_market'].values
excess_market_sq = df['excess_market'].values ** 2

Again using the formula $\hat{\beta} = (X'X)^{-1}X'y$ we obtain:

In [7]:
X2 = np.column_stack(
    (np.ones(len(df)), D_excess, nonD_excess, excess_market_sq))
beta_hat2 = np.linalg.inv(X2.T @ X2) @ (X2.T @ y)
print('Model 2 coefficients:\nalpha: ', beta_hat2[0], ' beta1: ', beta_hat2[1],
      '\nbeta2: ', beta_hat2[2], ' beta3: ', beta_hat2[3])

Model 2 coefficients:
alpha:  -0.010031847996995746  beta1:  1.6230560811213248 
beta2:  0.11105859440112376  beta3:  -6.095601716965916


### With builtin functions

In [8]:
df['Dt'] = (df['excess_market'] > 0).astype(int)
df['D_excess'] = df['Dt'] * df['excess_market']
df['nonD_excess'] = (1 - df['Dt']) * df['excess_market']
df['excess_market_sq'] = df['excess_market']**2

df.head()

,1-month Tbill,SP500,IBM,SP500_ret,IBM_ret,rft,excess_stock,excess_market,date,Dt,D_excess,nonD_excess,excess_market_sq
25,0.20,70.22,2.66,0.016650,-0.011152,0.0020,-0.013152,0.014650,1962-02-01,1,0.01465,0.000000,0.000215
26,0.20,70.29,2.64,0.000997,-0.007519,0.0020,-0.009519,-0.001003,1962-03-01,0,-0.00000,-0.001003,0.000001
27,0.22,68.05,2.25,-0.031868,-0.147727,0.0022,-0.149927,-0.034068,1962-04-01,0,-0.00000,-0.034068,0.001161
28,0.24,62.99,1.95,-0.074357,-0.133333,0.0024,-0.135733,-0.076757,1962-05-01,0,-0.00000,-0.076757,0.005892
29,0.20,55.63,1.68,-0.116844,-0.138462,0.0020,-0.140462,-0.118844,1962-06-01,0,-0.00000,-0.118844,0.014124


In [9]:
X3 = sm.add_constant(df[['D_excess', 'nonD_excess', 'excess_market_sq']])
model2 = sm.OLS(df['excess_stock'], X3).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:           excess_stock   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     48.87
Date:                Tue, 18 Nov 2025   Prob (F-statistic):           2.53e-26
Time:                        13:26:18   Log-Likelihood:                 546.66
No. Observations:                 347   AIC:                            -1085.
Df Residuals:                     343   BIC:                            -1070.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0100      0.006  

The second model allows different market exposures depending on whether excess market returns are positive or negative. The high value for beta1 relative to beta2 suggests IBM’s response to positive market movements is much larger than to negative movements. The negative beta3 indicates a possible concavity in the relationship, showing diminishing marginal response for larger market returns. <br>
$R^2$ has improved slightly from the first model, indicating a better fit.

## F Test

Here, we want to test the null hypothesis $H_0: \beta_1 = \beta_2$

### Manual computation

$\hat{y} = X\hat{\beta}$



(((($RSS = \sum_{i=1}^{n}(y_i - \hat{y_i})^2$))))

In [10]:
R= np.array([[0, 1, -1,0]])
r = 0

var1 = np.linalg.inv(X2.T @ X2)
var_R = np.linalg.inv(R @ var1 @ R.T)

e = df['excess_stock'].values - X2 @ beta_hat2
T = X2.shape[0]
e_eprime = (e.T @ e)/(T-4)

F_stats = ((R @ beta_hat2 - r)*var_R * (R @ beta_hat2 - r).T) / 1 / e_eprime
print('F statistic:', F_stats)

F statistic: [[5.76675586]]


In [18]:
from scipy import stats

stats.f.sf(F_stats, 1, T - 4)

array([[0.01686377]])

### With builtin functions

In [14]:
f_test_result = model2.f_test('D_excess = nonD_excess')
print(f_test_result)

<F test: F=5.766755863532396, p=0.01686377114880381, df_denom=343, df_num=1>


## T-test 

In [15]:
y_hat1 = X @ beta_hat
residuals1 = y - y_hat1
sigma2 = np.sum(residuals1 ** 2) / (len(y) - X.shape[1])
cov_beta_hat = sigma2 * np.linalg.inv(X.T @ X)
se_alpha = np.sqrt(cov_beta_hat[0, 0])
t_stat_alpha = alpha / se_alpha
print('t statistic for alpha:', t_stat_alpha)

t statistic for alpha: 0.9930805477699152


In [16]:
t_test_alpha = model1.t_test('const = 0')
print(t_test_alpha)

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0027      0.003      0.993      0.321      -0.003       0.008


In [17]:
t_test_alpha2 = model2.t_test('const = 0')
print(t_test_alpha2)

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0100      0.006     -1.757      0.080      -0.021       0.001


# Question 2

In [21]:
data = pd.read_excel('../data/data_coursework1_Q2.xlsx', sheet_name="final_m")

data['Y'] = (data['ESR'] > 0).astype(int)

In [22]:
original_vars = ['ESR', 'EIF', 'ERECB', 'EFX',
                 'EDY', 'ESP', 'UIF', 'URR', 'UDY', 'UOIL']

X_data = data[original_vars].shift(1).copy()

In [ ]:
data_clean = X_data.join(data['Y']).dropna()

Y = data_clean['Y'].values
X = data_clean[original_vars].values
X = np.c_[np.ones(X.shape[0]), X]

In [27]:
from scipy.stats import norm
from scipy.optimize import minimize

def neg_log_likelihood_probit(alpha, X, Y):
    """
    Calcule la Log-Vraisemblance Négative pour le modèle Probit.
    alpha: vecteur des paramètres (coefficients).
    X: matrice des variables explicatives.
    Y: vecteur de la variable dépendante binaire.
    """
    # Calcul de l'indice : I_t = alpha' * X_t
    index = X @ alpha

    # Calcul de la probabilité P_t = Phi(I_t)
    P = norm.cdf(index)

    # Assurer la stabilité numérique pour éviter log(0)
    P = np.clip(P, 1e-15, 1 - 1e-15)

    # Log-Vraisemblance (L(alpha))
    # l(alpha) = sum [ Y_t * log(P_t) + (1 - Y_t) * log(1 - P_t) ]
    log_likelihood = np.sum(Y * np.log(P) + (1 - Y) * np.log(1 - P))

    # Nous minimisons la Log-Vraisemblance Négative
    return -log_likelihood

In [29]:
initial_alpha = np.zeros(X.shape[1])

# Optimisation (minimisation) de la Log-Vraisemblance Négative
# 'BFGS' est un algorithme de quasi-Newton robuste pour l'optimisation non contrainte.
result = minimize(
    neg_log_likelihood_probit,
    initial_alpha,
    args=(X, Y),
    method='BFGS',
    options={'disp': True}
)

Optimization terminated successfully.
         Current function value: 176.930843
         Iterations: 31
         Function evaluations: 374
         Gradient evaluations: 34
